In [2]:
# Employee Data Processing Solution
# ------------------------------

# Import necessary libraries
import pandas as pd
import numpy as np
import os
import zipfile
import csv
import json
from IPython.display import display



In [4]:


# Task 1: Import Data
# ------------------------------
print("Task 1: Importing salary data...")

# Assuming the salary data is in a CSV file named 'salary_data.csv'
# If your data is in a different format or location, adjust this accordingly
try:
    # Try to import the data
    salary_data = pd.read_csv('salary_data.csv')
    print("Data imported successfully!")
    print(f"Data shape: {salary_data.shape}")
    display(salary_data.head())
except FileNotFoundError:
    print("Error: The salary data file was not found.")
    print("Creating sample data for demonstration purposes...")
    
    # Create sample data for demonstration
    salary_data = pd.DataFrame({
        'employee_id': [1001, 1002, 1003, 1004, 1005],
        'name': ['John Smith', 'Jane Doe', 'Robert Johnson', 'Emily Davis', 'Michael Wilson'],
        'department': ['Engineering', 'Marketing', 'Finance', 'HR', 'Engineering'],
        'position': ['Senior Developer', 'Marketing Manager', 'Financial Analyst', 'HR Specialist', 'Junior Developer'],
        'salary': [95000, 85000, 75000, 65000, 60000],
        'experience_years': [8, 6, 5, 4, 2],
        'email': ['john.smith@company.com', 'jane.doe@company.com', 'robert.j@company.com', 
                  'emily.davis@company.com', 'michael.w@company.com']
    })
    print("Sample data created.")
    display(salary_data.head())


Task 1: Importing salary data...
Data imported successfully!
Data shape: (312882, 9)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14172\2011536270.py:9: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  salary_data = pd.read_csv('salary_data.csv')


,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,Not Provided,567595.43,567595.43,2011
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,Not Provided,538909.28,538909.28,2011
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,Not Provided,335279.91,335279.91,2011
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,Not Provided,332343.61,332343.61,2011
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,Not Provided,326373.19,326373.19,2011


In [7]:

# Task 2: Create Employee Function
# ------------------------------
print("\nTask 2: Developing employee lookup function...")

def get_employee_details(employee_name, data=salary_data):
    """
    Returns the details of an employee based on their name.
    
    Args:
        employee_name (str): The name of the employee to look up
        data (DataFrame): The DataFrame containing employee data
        
    Returns:
        dict: A dictionary with the employee's details or an error message
    """
    try:
        # Convert input to lowercase for case-insensitive matching
        employee_name = employee_name.lower()
        
        # Find employees whose names match (case-insensitive)
        matches = data[data['EmployeeName'].str.lower() == employee_name]
        
        if matches.empty:
            return {"error": f"No employee found with name: {employee_name}"}
        
        # Convert the first match to a dictionary (in case of multiple matches with the same name)
        employee_details = matches.iloc[0].to_dict()
        return employee_details
    
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}



Task 2: Developing employee lookup function...


In [15]:

# Test the function
print("Testing employee lookup function:")
test_employee = "NATHANIEL FORD"
result = get_employee_details(test_employee)
print(f"Details for '{test_employee}':")
print(json.dumps(result, indent=2))

# Task 3: Data Processing with Dictionary
# ------------------------------
print("\nTask 3: Processing salary data using a dictionary...")

def process_salary_data(data=salary_data):
    """
    Process the salary data and return basic statistics using dictionaries.
    All values are converted to JSON-serializable types.
    
    Args:
        data (DataFrame): The DataFrame containing employee salary data
        
    Returns:
        dict: A dictionary with basic salary statistics
    """
    try:
        # Create a copy of the data to avoid modifying the original
        df = data.copy()
        
        # Replace 'Not Provided' with zeros across the DataFrame
        df = df.replace('Not Provided', 0)
        
        # Convert numeric columns to float
        numeric_columns = ['BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
        # Basic overall statistics - use float() to convert numpy types to Python types
        stats = {
            'total_employees': int(len(df)),
            'salary_stats': {
                'mean_base_pay': float(df['BasePay'].mean()),
                'median_base_pay': float(df['BasePay'].median()),
                'min_base_pay': float(df['BasePay'].min()),
                'max_base_pay': float(df['BasePay'].max()),
                'total_base_pay_sum': float(df['BasePay'].sum()),
            },
            'overtime_stats': {
                'mean_overtime': float(df['OvertimePay'].mean()),
                'median_overtime': float(df['OvertimePay'].median()),
                'max_overtime': float(df['OvertimePay'].max()),
                'employees_with_overtime': int((df['OvertimePay'] > 0).sum())
            },
            'total_compensation_stats': {
                'mean_total_pay': float(df['TotalPay'].mean()),
                'median_total_pay': float(df['TotalPay'].median()),
                'min_total_pay': float(df['TotalPay'].min()),
                'max_total_pay': float(df['TotalPay'].max()),
            }
        }
        
        # Add Benefits statistics if they're not all zeros/Not Provided
        if df['Benefits'].sum() > 0:
            stats['benefits_stats'] = {
                'mean_benefits': float(df['Benefits'].mean()),
                'median_benefits': float(df['Benefits'].median()),
                'employees_with_benefits': int((df['Benefits'] > 0).sum()),
                'percentage_with_benefits': float((df['Benefits'] > 0).sum() / len(df) * 100)
            }
        
        # Job title analysis - count employees per job title
        job_title_counts = df['JobTitle'].value_counts().to_dict()
        # Convert any numpy types in the dictionary
        job_title_counts = {k: int(v) for k, v in job_title_counts.items()}
        # Only include the top 10 job titles if there are many
        top_job_titles = dict(sorted(job_title_counts.items(), key=lambda x: x[1], reverse=True)[:10])
        stats['job_title_distribution'] = top_job_titles
        
        # Year analysis if multiple years exist
        if df['Year'].nunique() > 1:
            year_stats = {}
            for year in sorted(df['Year'].unique()):
                year_data = df[df['Year'] == year]
                year_stats[str(year)] = {
                    'employee_count': int(len(year_data)),
                    'mean_total_pay': float(year_data['TotalPay'].mean()),
                    'total_payroll': float(year_data['TotalPay'].sum())
                }
            stats['yearly_comparison'] = year_stats
        
        # Highest paid employees
        top_earners_df = df.nlargest(5, 'TotalPay')[['EmployeeName', 'JobTitle', 'TotalPay']]
        top_earners = []
        for _, row in top_earners_df.iterrows():
            top_earners.append({
                'EmployeeName': row['EmployeeName'],
                'JobTitle': row['JobTitle'],
                'TotalPay': float(row['TotalPay'])
            })
        stats['top_earners'] = top_earners
        
        # Pay distribution analysis
        pay_ranges = [
            (0, 50000),
            (50000, 100000),
            (100000, 200000),
            (200000, 300000),
            (300000, float('inf'))
        ]
        
        pay_distribution = {}
        for lower, upper in pay_ranges:
            if upper == float('inf'):
                label = f"${lower/1000:.0f}k+"
            else:
                label = f"${lower/1000:.0f}k-${upper/1000:.0f}k"
            
            count = len(df[(df['TotalPay'] >= lower) & (df['TotalPay'] < upper)])
            pay_distribution[label] = int(count)
            
        stats['pay_distribution'] = pay_distribution
        
        # Calculate the ratio of overtime to base pay for those with overtime
        overtime_workers = df[df['OvertimePay'] > 0]
        if len(overtime_workers) > 0:
            stats['overtime_analysis'] = {
                'avg_overtime_to_base_ratio': float((overtime_workers['OvertimePay'] / overtime_workers['BasePay']).mean()),
                'max_overtime_to_base_ratio': float((overtime_workers['OvertimePay'] / overtime_workers['BasePay']).max()),
                'overtime_percentage_of_total_compensation': float((df['OvertimePay'].sum() / df['TotalPay'].sum() * 100))
            }
        
        return stats
    
    except Exception as e:
        return {"error": f"An error occurred during data processing: {str(e)}"}

# Process the data
processed_salary_data = process_salary_data()
print("Salary data processed successfully:")
print(json.dumps(processed_salary_data, indent=2))



Testing employee lookup function:
Details for 'NATHANIEL FORD':
{
  "EmployeeName": "NATHANIEL FORD",
  "JobTitle": "GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY",
  "BasePay": 167411.18,
  "OvertimePay": 0.0,
  "OtherPay": 400184.25,
  "Benefits": "Not Provided",
  "TotalPay": 567595.43,
  "TotalPayBenefits": 567595.43,
  "Year": 2011
}

Task 3: Processing salary data using a dictionary...
Salary data processed successfully:
{
  "total_employees": 312882,
  "salary_stats": {
    "mean_base_pay": 69673.05060003453,
    "median_base_pay": 67645.97,
    "min_base_pay": -474.4,
    "max_base_pay": 592394.34,
    "total_base_pay_sum": 21799443417.840004
  },
  "overtime_stats": {
    "mean_overtime": 5668.911274442122,
    "median_overtime": 0.0,
    "max_overtime": 309481.03,
    "employees_with_overtime": 152762
  },
  "total_compensation_stats": {
    "mean_total_pay": 78802.64578777303,
    "median_total_pay": 74908.79000000001,
    "min_total_pay": -618.13,
    "max_total_pay": 5923

In [16]:

# Task 4: Error Handling
# ------------------------------
print("\nTask 4: Implementing error handling...")

def safe_employee_operation(func, *args, **kwargs):
    """
    A wrapper function that provides error handling for employee operations.
    
    Args:
        func: The function to execute
        *args: Positional arguments for the function
        **kwargs: Keyword arguments for the function
        
    Returns:
        The result of the function or an error message
    """
    try:
        return func(*args, **kwargs)
    except KeyError as e:
        return {"error": f"Key error: {str(e)}. The requested field does not exist."}
    except ValueError as e:
        return {"error": f"Value error: {str(e)}. Please check your input values."}
    except TypeError as e:
        return {"error": f"Type error: {str(e)}. Please check your input types."}
    except Exception as e:
        return {"error": f"An unexpected error occurred: {str(e)}"}



Task 4: Implementing error handling...


In [18]:

# Test error handling with valid and invalid inputs
print("Testing error handling:")

# Valid case
valid_result = safe_employee_operation(get_employee_details, "NATHANIEL FORD")
print("Valid case result:")
print(json.dumps(valid_result, indent=2))

# Invalid case - employee not found
invalid_result = safe_employee_operation(get_employee_details, "Nonexistent Person")
print("\nInvalid case result (employee not found):")
print(json.dumps(invalid_result, indent=2))


Testing error handling:
Valid case result:
{
  "EmployeeName": "NATHANIEL FORD",
  "JobTitle": "GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY",
  "BasePay": 167411.18,
  "OvertimePay": 0.0,
  "OtherPay": 400184.25,
  "Benefits": "Not Provided",
  "TotalPay": 567595.43,
  "TotalPayBenefits": 567595.43,
  "Year": 2011
}

Invalid case result (employee not found):
{
  "error": "No employee found with name: nonexistent person"
}


In [19]:

# Task 5: Export Employee Details
# ------------------------------
print("\nTask 5: Exporting employee details to CSV and creating a zipped folder...")

def export_employee_to_csv(employee_name, output_dir="Employee_Profile"):
    """
    Exports an employee's details to a CSV file and zips it into a folder.
    
    Args:
        employee_name (str): The name of the employee to export
        output_dir (str): The name of the output directory
        
    Returns:
        dict: A status message indicating success or failure
    """
    try:
        # Get employee details
        employee_details = get_employee_details(employee_name)
        
        if "error" in employee_details:
            return employee_details
        
        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Create a filename based on employee name
        filename = f"{employee_name.replace(' ', '_').lower()}_profile.csv"
        filepath = os.path.join(output_dir, filename)
        
        # Write to CSV
        with open(filepath, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            # Write header
            writer.writerow(employee_details.keys())
            # Write values
            writer.writerow(employee_details.values())
        
        # Create a zip file
        zip_filename = f"{output_dir}.zip"
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            # Add the CSV file to the zip
            zipf.write(filepath, os.path.basename(filepath))
        
        return {
            "status": "success",
            "message": f"Employee details exported to {filename} and zipped into {zip_filename}",
            "csv_path": filepath,
            "zip_path": zip_filename
        }
    
    except Exception as e:
        return {"error": f"Failed to export employee details: {str(e)}"}

# Export an employee's details
export_result = safe_employee_operation(export_employee_to_csv, "NATHANIEL FORD")
print("Export result:")
print(json.dumps(export_result, indent=2))




Task 5: Exporting employee details to CSV and creating a zipped folder...
Export result:
{
  "status": "success",
  "message": "Employee details exported to nathaniel_ford_profile.csv and zipped into Employee_Profile.zip",
  "csv_path": "Employee_Profile\\nathaniel_ford_profile.csv",
  "zip_path": "Employee_Profile.zip"
}


In [20]:

# Task 6: Unzip and Display Data with R
# ------------------------------
print("\nTask 6: R code to unzip the folder and display the data")

# Since we can't run R code directly in Python, we'll create an R script
r_script = '''
# R script to unzip and display employee data

# Load required libraries
library(readr)
library(dplyr)

# Function to unzip and display employee data
unzip_and_display_employee <- function(zip_file = "Employee_Profile.zip") {
  # Create a temporary directory
  temp_dir <- tempdir()
  
  # Unzip the file
  tryCatch({
    unzip(zip_file, exdir = temp_dir)
    cat("Successfully unzipped file to:", temp_dir, "\n")
  }, error = function(e) {
    stop("Error unzipping file: ", e$message)
  })
  
  # Find CSV files in the extracted directory
  csv_files <- list.files(temp_dir, pattern = "*.csv", full.names = TRUE)
  
  if (length(csv_files) == 0) {
    stop("No CSV files found in the unzipped directory")
  }
  
  # Read and display each CSV file
  for (file in csv_files) {
    cat("\nReading file:", basename(file), "\n")
    data <- read_csv(file)
    
    cat("Employee data:\n")
    print(data)
    
    # Optional: Display some statistics
    if ("salary" %in% colnames(data)) {
      cat("\nEmployee salary:", data$salary, "\n")
    }
    
    if ("department" %in% colnames(data)) {
      cat("Employee department:", data$department, "\n")
    }
  }
  
  return(invisible(NULL))
}

# Execute the function
unzip_and_display_employee()
'''



Task 6: R code to unzip the folder and display the data


In [21]:

# Save the R script to a file
r_script_path = "unzip_and_display_employee.R"
with open(r_script_path, 'w') as r_file:
    r_file.write(r_script)

print(f"R script saved to {r_script_path}")
print("To run the R script, use the following command in R or RStudio:")
print("source('unzip_and_display_employee.R')")

# Summary
# ------------------------------
print("\nAssignment Summary:")
print("1. Data Import: ✓ Completed")
print("2. Employee Function: ✓ Completed")
print("3. Dictionary Processing: ✓ Completed")
print("4. Error Handling: ✓ Completed")
print("5. Export to CSV and Zip: ✓ Completed")
print("6. R Script for Unzipping: ✓ Completed")
print("\nAll tasks have been completed successfully!")


R script saved to unzip_and_display_employee.R
To run the R script, use the following command in R or RStudio:
source('unzip_and_display_employee.R')

Assignment Summary:
1. Data Import: ✓ Completed
2. Employee Function: ✓ Completed
3. Dictionary Processing: ✓ Completed
4. Error Handling: ✓ Completed
5. Export to CSV and Zip: ✓ Completed
6. R Script for Unzipping: ✓ Completed

All tasks have been completed successfully!
